In [1]:
import cv2 as cv
import numpy as np

In [2]:
cap = cv.VideoCapture('videos/planet.mp4')

while(cap.isOpened()):
  # ret = a boolean return value from getting the frame, frame = the current frame being projected in the video
  ret, frame = cap.read()

  if not ret:
    print('No frames grabbed!')
    break

  cv.imshow('frame',frame)
  if cv.waitKey(10) & 0xFF == ord('q'):
    break

cap.release()
cv.destroyAllWindows()

No frames grabbed!


In [3]:
cap = cv.VideoCapture('videos/planet.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 300,
                       qualityLevel = 0.2,
                       minDistance = 2,
                       blockSize = 7 )

# Parameters for Lucas Kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Variable for color to draw optical flow track
# color = (255, 255, 0)
color = np.random.randint(0, 255, (100, 3))

# ret : boolean return value from getting the frame ; first_frame : first frame in the entire video sequence
ret, first_frame = cap.read()
# Converts frame to grayscale because we only need the luminance channel for detecting edges - less computationally expensive
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
prev = cv.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(first_frame)

while(cap.isOpened()):
  # ret = a boolean return value from getting the frame, frame = the current frame being projected in the video
  ret, frame = cap.read()

  if not ret:
    print('No frames grabbed!')
    break

  frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
  prev = cv.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)
  # calculate optical flow
  next, status, error = cv.calcOpticalFlowPyrLK(prev_gray, frame_gray, prev, None, **lk_params)

  # Selects good feature points for previous position
  if next is not None:
    good_new = next[status==1].astype(int)
    good_old = prev[status == 1].astype(int)

  # Draws the optical flow tracks
  for i, (new, old) in enumerate(zip(good_new, good_old)):
    # Returns a contiguous flattened array as (x, y) coordinates for new point
    a, b = new.ravel()
    # Returns a contiguous flattened array as (x, y) coordinates for old point
    c, d = old.ravel()
    # Draws line between new and old position
    mask = cv.line(mask, (a, b), (c, d), color[i].tolist(), 2)
    # Draws filled circle (thickness of -1) at new position
    frame = cv.circle(frame, (a, b), 3, color[i].tolist(), -1)
  # Overlays the optical flow tracks on the original frame
  output = cv.add(frame, mask)
  # Updates previous frame and good feature points
  prev_gray = frame_gray.copy()
  prev = good_new.reshape(-1, 1, 2)

  # Opens a new window and displays the output frame
  cv.imshow('output',output)
  if cv.waitKey(10) & 0xFF == ord('q'):
    break

# Frees up resources and closes all windows
cap.release()
cv.destroyAllWindows()

No frames grabbed!
